In [3]:
import os
import pandas as pd
import numpy as np
!pip3 install Bio

import Bio
from Bio import Entrez
!pip3 install ncbi-datasets-pylib
import ncbi.datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 29.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
'''
Summary:
Using the data from the PDF which had viable assemblies (see supplementary info),
we mapped each entry to its corresponding SRR run accessions. BioSamples of each
entry in the data from the PDF were put through SRA Run Selector to find corresponding 
SRR accessions. This notebook does the aforementioned mapping and writes all 
entries with their associated SRR runs to CSV files which are specific to each 
BioSample.

The 200 Mammals Project (PRJNA399431) only has 127 SRA projects, hence the number 
of csv files I was able to make


TODO:
- Need to pull all runs from each biosample
    - Currently I was just pulling SRS from each Genbank accession, unless that's exactly
        what I should be doing?
        - SRS (from SRA search on ncbi) leads to SRRs
        - In the example Erik sent, each CSV entry's only unique field is Run

Notes:
- SRA Run Selector search on all Biosamples returned same number of results as SRS
- Biosample search can find SRA
- Genbank Accession can find BioSample, BioProject
- SRA can find Project
- Project can find Genbank
- If there are multiple entries from the same Bioproject, there may be a chance that 
    they all don't have SRA Experiments (ex: Monodon monoceros (male) from PRJNA399349)

- xcel with samples: https://docs.google.com/spreadsheets/d/1rMadjpp_P87Beop3otg3N3hirS1CdDKq/edit#gid=1033975903

- NCBI datasets notebook: https://github.com/ncbi/datasets/blob/master/examples/jupyter/ncbi-datasets-pylib/ncbi-datasets-assembly.ipynb
- Info on SRA meanings: https://linsalrob.github.io/ComputationalGenomicsManual/Databases/SRA.html#:~:text=A%20submission%20is%20a%20package,or%20more%20runs%20(SRR).
- Batch Entrez can give me SRRs: https://www.ncbi.nlm.nih.gov/sites/batchentrez
- SRA run selector: https://www.ncbi.nlm.nih.gov/Traces/study/?WebEnv=MCID_62e06a2c2a3de3782a696c7c&query_key=69
'''
os.listdir()

['.config', 'drive', 'sample_data']

In [22]:
pd.set_option('display.max_columns', None) # Allows all cols to be shown
h1 = "Set,Species,Common Name,Order,Family,IUCN,Biosample".split(",")
h2 = "Provider Institution\tProvider contact\tSequencing location \tGenbank accession\tContig N50 (bp)**\tScaffold N50 (bp)**\tSize (Gb; contigs>1kb)**\tMean base quality** \tCoverage**\tBusco Complete (n=4104)\tBUSCO Single Copy\tBUSCO Duplicated\tBUSCO Fragmented\tBUSCO Missing".split("\t")
categories = h1+h2
path = "drive/MyDrive/data/msmc-data/MSMC-Exploratory-Analysis/mammal-analysis/"
fileName = "mammal_data.tsv"
os.listdir(path)
df = pd.read_csv(path+fileName, sep="\t", header=None)
df.columns = categories
df.to_csv(path+"original_data/mammal_metadata.csv")

pdf_df = df[["Biosample", "Genbank accession", "Species"]]
pdf_df["SRS"] = "N/A"
pdf_df["BioProject"] = "N/A"

allBSdf = pdf_df.dropna(subset=["Biosample"])
 # Keeps entries with minimum required data
pdf_df = pdf_df.dropna(subset=["Genbank accession"]) # Keeps entries with minimum required data
pdf_df = pdf_df.set_index("Genbank accession")

pdf_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Biosample,Species,SRS,BioProject
Genbank accession,,,,
GCA_004363575.1,SAMN07678062,Solenodon paradoxus,N/A,N/A
GCA_004027635.1,SAMN07678044,Crocidura indochinensis,N/A,N/A
GCA_004024925.1,SAMN07678045,Scalopus aquaticus,N/A,N/A
GCA_004024945.1,SAMN07678046,Uropsilus gracilis,N/A,N/A
GCA_004023825.1,SAMN07678118,Vulpes lagopus,N/A,N/A
...,...,...,...,...
GCA_004027255.2,SAMN07678019,Galeopterus variegatus,N/A,N/A
GCA_004026925.2,SAMN07678107,Procavia capensis,N/A,N/A
GCA_004027065.2,SAMN07678089,Hippopotamus amphibius,N/A,N/A


In [23]:
from pandas.core.indexes.multi import sparsify_labels
'''
Code cell adds SRS and BioProject data to pdf_df
'''
api_instance = ncbi.datasets.GenomeApi(ncbi.datasets.ApiClient()) # Python API for NCBI datasets
genbankList = pdf_df.index # List of genbank accessions
for idx, asm_acc in enumerate(genbankList):
    # print(idx)
    genome_summary = api_instance.assembly_descriptors_by_accessions([asm_acc])
    # print(f"Number of assemblies: {genome_summary.total_count}" )
    try:
        srs        = genome_summary["assemblies"][0]["assembly"]["biosample"]["sample_ids"][1]["value"]
    except:
        srs = "N/A"
        print(f"Passed on SRA: entry {idx} ({asm_acc})")
    try:
        bioProject = genome_summary["assemblies"][0]["assembly"]["biosample"]["bioprojects"][0]["accession"] # might not be a thorough check
    except:
        try:
            bioProject = genome_summary["assemblies"][0]["assembly"]["bioproject_lineages"][0]["bioprojects"][0]["accession"] # Made for exception with GCF_000313985.2
        except:
            bioProject = "N/A"
            print(f"Passed on project: entry {idx} ({asm_acc})")
    pdf_df.loc[asm_acc, "SRS"] = srs
    pdf_df.loc[asm_acc, "BioProject"] = bioProject


Passed on SRA: entry 36 (GCA_004026685.1)
Passed on SRA: entry 94 (GCA_007922755.1)
Passed on SRA: entry 112 (GCA_004027275.1)
Passed on SRA: entry 113 (GCA_004024665.1)
Passed on SRA: entry 140 (GCA_004765945.2)
Passed on project: entry 140 (GCA_004765945.2)


In [24]:
'''
There are some genbank accessions which share a Biosample
- 5 genbank accessions were filtered out because they lacked an SRA project in NCBI DB
    - Mentioned in supplementary info (Set C: Failed assemblies)
- 8 of these Biosamples contain 2 Genbank Accessions
    - These correspond to the Genbank Accessions with .2
    - These Biosamples have a 1:1 relationship with the SRS accessions
'''

pdf_df = pdf_df.dropna(subset=["BioProject"])
pdf_df = pdf_df.dropna(subset=["Biosample"])
pdf_df = pdf_df.dropna(subset=["SRS"])
pdf_df.at["GCA_004027145.1", "Species"] = "Daubentonia madagascariensis" # Necessary fix, PDF data was misread causing a small typo of the species name for this gb acc
pdf_df

,Biosample,Species,SRS,BioProject
Genbank accession,,,,
GCA_004363575.1,SAMN07678062,Solenodon paradoxus,SRS9004990,PRJNA399371
GCA_004027635.1,SAMN07678044,Crocidura indochinensis,SRS3678510,PRJNA399360
GCA_004024925.1,SAMN07678045,Scalopus aquaticus,SRS3678531,PRJNA399361
GCA_004024945.1,SAMN07678046,Uropsilus gracilis,SRS3678540,PRJNA399362
GCA_004023825.1,SAMN07678118,Vulpes lagopus,SRS3678542,PRJNA399424
...,...,...,...,...
GCA_004027255.2,SAMN07678019,Galeopterus variegatus,SRS3678514,PRJNA399345
GCA_004026925.2,SAMN07678107,Procavia capensis,SRS3678530,PRJNA399414
GCA_004027065.2,SAMN07678089,Hippopotamus amphibius,SRS9006326,PRJNA399399


In [25]:
[len(pdf_df[subset].unique()) for subset in pdf_df] # Number of unique entries in columns (except GB Accessions col which is both the index and unique)

[132, 132, 129, 132]

In [26]:
'''
Using the list of Biosamples given from original PDF table metadata, I made a csv (allBioSamples_SraRunTable.txt)
list of Biosample entries, put em in the search bar for SRA Run Selector and popped
out all this meta data which I represent as a df

Now I gotta extract: Run, Project,

Steps:
- Drop all without BioProject
- Drop all with a non SRR run name
- Count duplicate samples
    - Somehow partition dups under Biosamples

- Make a DF with all relevant info
- Make a dict where Biosamples are keys and values are lists containing corresponding SRR runs

Have:
- Biosample, Genbank Acc
Need:
- Run, BioProject, , 
- Potentially Library Name
'''

allBS_SraRun_df = pd.read_csv(path+"allBioSamples_SraRunTable.txt") # Read in data table from SRA Run Selector findings on pdf biosamples
print(allBS_SraRun_df.shape)
allBS_SraRun_df = allBS_SraRun_df.dropna(subset=["BioProject"]) # Remove all entries without a BioProject
print(allBS_SraRun_df.shape)
allBS_SraRun_df = allBS_SraRun_df.loc[[True if "SRR" in entry else False for entry in allBS_SraRun_df["Run"]]] # Select entries which contain an SRR
print(allBS_SraRun_df.shape)
allBS_SraRun_df = allBS_SraRun_df.dropna(subset=["BioSample"]) # Drop all SRs which don't have a biosample
print(allBS_SraRun_df.shape)

(4224, 91)
(3156, 91)
(179, 91)
(176, 91)


In [27]:
'''
Try to find all SRR entries that share a biosample
'''
allBS_SraRun_df

,Run,Assay Type,Center Name,Consent,Experiment,Instrument,LibraryLayout,LibrarySelection,LibrarySource,Platform,ReleaseDate,ENA-FIRST-PUBLIC (run),ENA-LAST-UPDATE (run),Library Name,BioProject,SRA Study,DATASTORE filetype,DATASTORE provider,DATASTORE region,AssemblyName,AvgSpotLen,Bases,Bytes,BioSample,Isolate,Organism,Sample Name,sex,Age,BioSampleModel,dev_stage,tissue,BIOMATERIAL_PROVIDER,Cultivar,BREED,Ecotype,strain,flowcell_barcode (run),geographic_location_(country_and/or_sea,region),instrument_name (run),lane (run),material_type (exp),project (exp),project (run),run_barcode (run),run_name (run),sample_name,work_request (exp),work_request (run),sample_type (exp),gssr_id (exp),gssr_id (run),lsid (exp),lsid (run),gssr_ids (exp),gssr_ids (run),lsids (exp),lsids (run),artic_primer_version (exp),sub_species,Barcode (exp),center_name (exp),center_project_name (exp),extraction_robot (exp),extractionkit_lot (exp),linker (exp),mastermix_lot (exp),orig_name (exp),pcr_primers (exp),plating (exp),primer_date (exp),primer (exp),primer_plate (exp),processing_robot (exp),project_name (exp),run_center (exp),run_date (exp),run_prefix (exp),runid (exp),sample_plate (exp),sequencing_meth (exp),specimen_voucher,target_gene (exp),target_subfragment (exp),tm1000_8_tool (exp),tm300_8_tool (exp),tm50_8_tool (exp),water_lot (exp),well_description (exp),well_id (exp)
0,SRR14569519,WGS,BROAD INSTITUTE,public,SRX10912773,Illumina HiSeq 2500,PAIRED,RANDOM,GENOMIC,ILLUMINA,2021-05-17T00:00:00Z,NaN,NaN,CatWag_BS18,PRJNA399378,SRP320136,"bam,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",NaN,500.0,1.585040e+11,9.835480e+10,SAMN07678067,BS18,Catagonus wagneri,CatWag_1_DISCOVAR,missing,not collected,Model organism or animal,adult,not applicable,"Dr. Oliver Ryder\, San Diego Zoo Institute for...",not applicable,not applicable,not applicable,not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SRR14572003,WGS,BROAD INSTITUTE,public,SRX10915222,Illumina HiSeq 2500,PAIRED,RANDOM,GENOMIC,ILLUMINA,2021-05-29T00:00:00Z,NaN,NaN,MosMos_BS20,PRJNA399405,SRP320198,"bam,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",NaN,500.0,1.696866e+11,1.013975e+11,SAMN07678096,BS20,Moschus moschiferus,MosMos_1_DISCOVAR,missing,not collected,Model organism or animal,adult,not applicable,"Dr. Oliver Ryder\, San Diego Zoo Institute for...",not applicable,not applicable,not applicable,not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SRR14572852,WGS,BROAD INSTITUTE,public,SRX10916071,Illumina HiSeq 2500,PAIRED,RANDOM,GENOMIC,ILLUMINA,2021-05-29T00:00:00Z,NaN,NaN,HipAmp_BS01,PRJNA399399,SRP320223,"bam,sra","gs,ncbi,s3","gs.US,ncbi.public,s3.us-east-1",NaN,500.0,2.062613e+11,1.309261e+11,SAMN07678089,BS01,Hippopotamus amphibius,HipAmp_1_DISCOVAR,missing,not collected,Model organism or animal,adult,not applicable,"Dr. Oliver Ryder\, San Diego Zoo Institute for...",not applicable,not applicable,not applicable,not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRR11427202,WGS,BROAD INSTITUTE,public,SRX8005550,Illumina HiSeq 2500,PAIRED,RANDOM,GENOMIC,ILLUMINA,2020-03-26T00:00:00Z,NaN,NaN,AntAmePen_BS35,PRJNA399373,SRP254103,"bam,sra","gs,s3","gs.US,s3.us-east-1",NaN,500.0,1.189613e+11,7.242393e+10,SAMN07678063,BS35,Antilocapra americana,AntAmePen_1_DISCOVAR,missing,not collected,Model organism or animal,adult,not applicable,"Dr. Oliver Ryder\, San Diego Zoo Institute for...",not applicable,not applicable,not applicable,not applicable,NaN,NaN,NaN

In [28]:
len(allBS_SraRun_df["BioSample"].unique().tolist())

128

In [29]:
'''
Keys found from SRA Run Selector output given Biosamples from PDF
Biosample to SRR dict 

- It appears that SAMN00216137 which has 49 of associated runs is not found in PDF
'''
bs2srr_runselector_dict = dict()
for samp in allBS_SraRun_df["BioSample"].unique().tolist():
    bs2srr_runselector_dict[samp] = []
for idx in allBS_SraRun_df.index:
    row = allBS_SraRun_df.loc[idx]
    bs2srr_runselector_dict[row["BioSample"]].append(row["Run"])
bs2srr_runselector_dict

{'SAMN00216137': ['SRR317809',
  'SRR278581',
  'SRR278582',
  'SRR278583',
  'SRR278584',
  'SRR107098',
  'SRR107099',
  'SRR107100',
  'SRR107101',
  'SRR107609',
  'SRR107610',
  'SRR107611',
  'SRR107612',
  'SRR107613',
  'SRR107614',
  'SRR107615',
  'SRR107616',
  'SRR107617',
  'SRR107618',
  'SRR107619',
  'SRR107620',
  'SRR107621',
  'SRR107622',
  'SRR107623',
  'SRR107624',
  'SRR107625',
  'SRR107626',
  'SRR107627',
  'SRR107628',
  'SRR107629',
  'SRR107630',
  'SRR107631',
  'SRR107632',
  'SRR107633',
  'SRR107634',
  'SRR107635',
  'SRR107636',
  'SRR107637',
  'SRR107638',
  'SRR107639',
  'SRR107640',
  'SRR107641',
  'SRR107642',
  'SRR107643',
  'SRR107644',
  'SRR107645',
  'SRR107646',
  'SRR107647',
  'SRR107648'],
 'SAMN07678014': ['SRR7637815'],
 'SAMN07678015': ['SRR7637808'],
 'SAMN07678016': ['SRR14571988'],
 'SAMN07678017': ['SRR7637804'],
 'SAMN07678018': ['SRR7704810'],
 'SAMN07678019': ['SRR7704812'],
 'SAMN07678020': ['SRR7637816'],
 'SAMN07678021':

In [30]:
pdf_biosamples = pdf_df["Biosample"].tolist() # From original data from PDF

runAcc = allBS_SraRun_df[["BioSample", "Run", "BioProject", "Organism"]] # SRA Run Selector entries which include Biosamples found in PDF
runAcc = runAcc.iloc[[True if sample in pdf_biosamples else False for sample in runAcc["BioSample"].tolist()]] # Now keep all entries which contain a recorded biosample

'''
!!! Info on 200 Mammals sequencing project (BIG) : https://www.ncbi.nlm.nih.gov/bioproject/312960 !!!
Issue:
Ambiguity of refGenome/genbank accession to Run/SRR. Really all that's left now
    - I could cheese around this by doing an NCBI datasets query on organism and
        take the Genbank accession with the assumption that the samples in this paper are
        related to Ref genomes
'''


runAcc["LibraryName"] = runAcc["BioSample"]
runAcc["refGenome"] = "N/A"
runAcc

,BioSample,Run,BioProject,Organism,LibraryName,refGenome
0,SAMN07678067,SRR14569519,PRJNA399378,Catagonus wagneri,SAMN07678067,N/A
1,SAMN07678096,SRR14572003,PRJNA399405,Moschus moschiferus,SAMN07678096,N/A
2,SAMN07678089,SRR14572852,PRJNA399399,Hippopotamus amphibius,SAMN07678089,N/A
3,SAMN07678063,SRR11427202,PRJNA399373,Antilocapra americana,SAMN07678063,N/A
4,SAMN07678078,SRR11430189,PRJNA399388,Diceros bicornis,SAMN07678078,N/A
...,...,...,...,...,...,...
171,SAMN00216137,SRR107644,PRJNA12590,Echinops telfairi,SAMN00216137,N/A
172,SAMN00216137,SRR107645,PRJNA12590,Echinops telfairi,SAMN00216137,N/A
173,SAMN00216137,SRR107646,PRJNA12590,Echinops telfairi,SAMN00216137,N/A
174,SAMN00216137,SRR107647,PRJNA12590,Echinops telfairi,SAMN00216137,N/A


In [31]:
[len(runAcc[subset].unique()) for subset in runAcc] # List of number of unique vals for each column in runAcc

[128, 176, 128, 128, 128, 1]

In [32]:
'''
Check if biosamples from Run Selector Entries have 1:1 mapping to PDF Biosamples

All of these biosamples (symmetric diff) belong to pdf_df
This is because relevan_SraRun is built from a collection of SRR entries which 
are only a subset of the BioSamples in pdf_df
'''
biosamples_sym_diff = set(runAcc["BioSample"].unique()).symmetric_difference(set(pdf_df["Biosample"].unique()))
print(biosamples_sym_diff) # Empty set means that all samples are shared


{'SAMN07678092', 'SAMN07678104', 'SAMN07678024', 'SAMN07678083'}


In [16]:
'''
Make a df for each unique BioSample from the runAcc OR split pdf_df into 
- Each GB acc which is in a 2 part series shares Pretty much everything
    - Might just keep the .2 GB accessions since they seem to be the updated versions
        - .2 GB accessions are from Set B mentioned by the supplementary info
            - They are like .1 but with upgraded contiguity

STEPS:
1. Remove .1 accessions from pdf_df if there is a .2 associated with it
2. With df of latest GB Acc's mentioned in PDF, form df between PDF and SRA Run Selector findings
    - Genbank Accessions from PDF seem to only have 1 SRR run each, making mapping possible
        - Each BioSample and BioProject seem to correspond to 1 Genbank Accession from the PDF
        - Each SRR seems specific to each Genbank Accession with the exception of 1 GCF accession (refseq must've been included by accident)
    - Species names differ slightly, species names in PDF are more specific

'''
# Step 1
gb_acc_kernels = {acc[:-2] : []for acc in pdf_df.index} 
for acc in pdf_df.index:
    gb_acc_kernels[acc[:-2]].append(acc)
best_genbank_accessions = [val[-1] if pdf_df.loc[val[-1]]["BioProject"] != "N/A" else val[0] for val in gb_acc_kernels.values()  ] # Make list of most up to date GB accessions (use .2 if available)
pdf_df_best = pdf_df.loc[best_genbank_accessions] # Create df with most up to date GB accessions
# pdf_df_best
# Step 2
for idx_i in runAcc.index: # assign proper refGenome and Organism to runAcc df
    row_i = runAcc.iloc[idx_i]
    for idx_j in pdf_df_best.index:
        row_j = pdf_df_best.loc[idx_j]
        if row_i["BioSample"] == row_j["Biosample"] and row_i["BioProject"] == row_j["BioProject"]: # Rows are a match
            # print(row_i)
            # print(row_j)
            runAcc.at[idx_i, "refGenome"] = idx_j
            runAcc.at[idx_i, "Organism"] = row_i["Organism"]
# By now runAcc should contain all the info we are looking for, now we just need to partition it by BioSample
runAcc = runAcc[["BioSample","LibraryName","refGenome","Run","Organism","BioProject"]]

# Step 2 Continued: Actully turning DF's into a format fit for CSVs
bioSampleList = list(runAcc["BioSample"].unique())
csv_df_precursor_collection = dict()
for bs in bioSampleList:
    csv_df_precursor_collection[bs] = runAcc.loc[runAcc["BioSample"] == bs]

# Write df's to CSVs
outpath = path+"CSVs/"
for key in bioSampleList:
    fileName = key + ".csv"
    csv_df_precursor_collection[key].to_csv(path_or_buf=outpath+fileName, index=False)

In [17]:
runAcc

,BioSample,LibraryName,refGenome,Run,Organism,BioProject
0,SAMN07678067,SAMN07678067,GCA_004024745.2,SRR14569519,Catagonus wagneri,PRJNA399378
1,SAMN07678096,SAMN07678096,GCA_004024705.2,SRR14572003,Moschus moschiferus,PRJNA399405
2,SAMN07678089,SAMN07678089,GCA_004027065.2,SRR14572852,Hippopotamus amphibius,PRJNA399399
3,SAMN07678063,SAMN07678063,GCA_004027515.2,SRR11427202,Antilocapra americana,PRJNA399373
4,SAMN07678078,SAMN07678078,GCA_004027315.2,SRR11430189,Diceros bicornis,PRJNA399388
...,...,...,...,...,...,...
171,SAMN00216137,SAMN00216137,GCF_000313985.2,SRR107644,Echinops telfairi,PRJNA12590
172,SAMN00216137,SAMN00216137,GCF_000313985.2,SRR107645,Echinops telfairi,PRJNA12590
173,SAMN00216137,SAMN00216137,GCF_000313985.2,SRR107646,Echinops telfairi,PRJNA12590
174,SAMN00216137,SAMN00216137,GCF_000313985.2,SRR107647,Echinops telfairi,PRJNA12590


In [18]:
[len(runAcc[col].unique()) for col in runAcc]

[128, 128, 128, 176, 128, 128]

In [19]:
# '''
# SRS to Genbank accession dict
# '''
# srs2gb_dict = dict()
# for samp in pdf_df["SRS"].unique().tolist():
#     srs2gb_dict[samp] = []
# for idx in pdf_df.index:
#     row = pdf_df.loc[idx]
#     srs2gb_dict[row["SRS"]].append(idx)
# # srs2gb_dict

# '''
# SRS to Biosample dict 
# '''
# srs2bs_dict = dict()
# for samp in pdf_df["SRS"].unique().tolist():
#     srs2bs_dict[samp] = []
# for idx in pdf_df.index:
#     row = pdf_df.loc[idx]
#     srs2bs_dict[row["SRS"]].append(pdf_df.loc[idx]["Biosample"])
# # srs2bs_dict

# '''
# BP to GB dict 
# '''
# bp2gb_dict = dict()
# for samp in pdf_df["BioProject"].unique().tolist():
#     bp2gb_dict[samp] = []
# for idx in pdf_df.index:
#     row = pdf_df.loc[idx]
#     bp2gb_dict[row["BioProject"]].append(idx)
# # bp2gb_dict

In [20]:
# '''
# With Bio.Entrez, I can search the following tables: 
# - https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly
# - https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/

# Bio.Entrez cookbook: http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec143

# Erik's resource: https://www.bioinfo.help/bio-search.html
# '''
# x = Entrez.efetch(db="nuccore", id="GCA_004027065.2", rettype="gb", retmode="text") # example
# # Can find longitude and latitude data
# for line in x:
#     print(line)

# '''
# Biosample to Genbank accession dict
# '''
# bs2gb_dict = dict()
# for samp in pdf_df["Biosample"].unique().tolist():
#     bs2gb_dict[samp] = []
# for idx in pdf_df.index:
#     row = pdf_df.loc[idx]
#     bs2gb_dict[row["Biosample"]].append(idx)
# bs2gb_dict